# Trabajo Final - Lenguaje de Programación 2
### Título del Proyecto
**Sistema de Monitoreo y Comparación de Precios de Hardware (Smarth Shop)**

### Nota

Todo el desarrollo del proyecto se ha realizado respetando las directivas del archivo robots.txt y los términos de servicio de las páginas web y APIs consultadas. Asimismo, se implementaron tiempos de espera (delays) entre las solicitudes para evitar la sobrecarga de los servidores externos y asegurar una extracción responsable de la información.

### Introducción y Relevancia del Proyecto

En un contexto económico marcado por la inflación y la constante variación de precios, los consumidores peruanos enfrentan dificultades para identificar ofertas reales en productos tecnológicos como laptops y componentes de hardware. Muchas veces, las promociones mostradas en tiendas virtuales no reflejan un verdadero ahorro, lo que genera desinformación y decisiones de compra poco óptimas.

El proyecto **Smart Shop** surge como una solución tecnológica orientada a centralizar, comparar y normalizar precios de productos de hardware provenientes de distintas tiendas, permitiendo a los usuarios identificar el precio más bajo disponible en el mercado al momento de la consulta. De esta manera, el sistema contribuye al ahorro económico, fomenta decisiones de compra informadas y protege el poder adquisitivo de personas que dependen de la tecnología para estudiar, trabajar o emprender.

### Objetivos del Proyecto
#### Objetivo General

Desarrollar un sistema automatizado en Python que permita monitorear y comparar precios de productos tecnológicos entre diferentes retailers, presentando la información de forma estructurada y estandarizada.

#### Objetivos Específicos

- Desarrollar un bot en Python capaz de extraer diariamente precios de laptops y componentes de hardware desde tiendas reconocidas.
- Implementar la normalización de moneda (USD a PEN) en tiempo real mediante el consumo de una API de tipo de cambio.
- Generar un dataset estructurado en formato CSV que permita analizar y visualizar la dispersión de precios de un mismo producto.
- Facilitar la identificación del producto más económico disponible en la fecha de ejecución del sistema.

### Fuentes de Datos y Estrategia de Extracción

El proyecto utiliza **tres tipos de fuentes de datos**, cumpliendo con los requisitos del curso y aplicando distintas técnicas de adquisición de información:

#### Fuente 1: Web Scraping Estático
- **Amazon**
- Se emplea la librería **BeautifulSoup** para extraer información de catálogos de laptops.
- Los datos recolectados incluyen: nombre del producto, precio, descripción y vendedor.
- Esta fuente permite obtener información estructurada directamente desde el HTML estático.

#### Fuente 2: Web Scraping Dinámico
- **Plaza Vea, Coolbox y Falabella**
- Se utiliza **Selenium** para renderizar contenido dinámico generado mediante JavaScript.
- Se extraen precios y ofertas exclusivas disponibles únicamente en la web.
- Esta técnica permite acceder a información que no está disponible mediante scraping tradicional.

#### Fuente 3: API Pública
- **ExchangeRate-API**
- Se consume una API REST para obtener el tipo de cambio actualizado entre dólares estadounidenses (USD) y soles peruanos (PEN).
- Esta información es clave para estandarizar todos los precios recolectados y permitir una comparación precisa entre tiendas.

### Integración y Tratamiento de Datos

Una vez obtenidos los datos desde las distintas fuentes, el sistema realiza un proceso de integración y limpieza que incluye:
- Conversión de todos los precios a moneda local (PEN).
- Normalización de nombres de productos para facilitar la comparación.
- Eliminación de registros duplicados.
- Manejo de valores nulos o inconsistentes.

El resultado de este proceso es un dataset limpio y estructurado, listo para su análisis posterior.

### Producto Final

El producto final del proyecto consiste en:
- Un **repositorio en GitHub** que contiene todo el código fuente, documentado y organizado por módulos.
- Un **archivo CSV/Excel** que presenta la comparación de precios de los productos analizados.
- Un reporte que permite identificar claramente cuál es el producto más barato del mercado en la fecha de ejecución del sistema.

Este sistema puede servir como base para futuras extensiones, como visualizaciones interactivas, alertas de precios o una aplicación web orientada al consumidor final.

### Tecnologías Utilizadas
- **Lenguaje:** Python 3.x
- **Librerías principales:**
    - Requests
    - BeautifulSoup4
    - Selenium
    - Pandas

- **Formato de salida:** CSV / Excel
- **Control de versiones:** Git y GitHub

### Importación de librerías necesarias para la extracción, procesamiento y almacenamiento de datos

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import time
import os

### Configuración de parámetros para Amazon y definición de función de limpieza de precios
En esta sección se definen los parámetros base para realizar el scraping de productos desde Amazon, incluyendo el término de búsqueda, la cantidad de páginas a recorrer y las cabeceras HTTP necesarias para simular un navegador real. Asimismo, se implementa una función auxiliar para la limpieza y normalización de precios, la cual será utilizada en etapas posteriores del proceso de extracción y análisis de datos.

In [ ]:
# --- 1. CONFIGURACIÓN INICIAL PARA AMAZON ---
TERMINO_BUSQUEDA = "computadores"
URL_INICIAL = f"https://www.amazon.com/s?k={TERMINO_BUSQUEDA}" 
NUMERO_DE_PAGINAS = 5  # Extracción de 5 páginas


# Headers avanzados para simular un navegador genuino y reducir
# la probabilidad de bloqueos durante el scraping
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Referer': 'https://www.amazon.com/', 
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'es-ES,es;q=0.9',
    'Cookie': 'custom_cookie=true',
    'Connection': 'keep-alive',
}


# --- FUNCIÓN AUXILIAR PARA LIMPIEZA DE PRECIOS ---
# Esta función permite transformar los precios extraídos desde
# la web (en formato texto) a un formato numérico estándar,
# eliminando símbolos monetarios, separadores de miles y texto
# irrelevante, con el fin de permitir cálculos y comparaciones.
def limpiar_precio(precio_str):
    """
    Limpia y normaliza el precio extraído como texto, eliminando
    símbolos monetarios, texto adicional y separadores innecesarios,
    garantizando un formato numérico adecuado para análisis posterior.

    Parámetros:
    precio_str (str): Precio en formato texto obtenido durante el scraping.

    Retorna:
    float | None: Precio convertido a tipo float o None si la conversión falla.
    """
    if isinstance(precio_str, str):
        # 1. Eliminar caracteres no deseados y texto de referencia
        precio_limpio = precio_str.replace('\u00a0', ' ').replace('USD', '').replace('PEN', '').replace('S/', '').replace('$', '').replace('PVPR:', '').replace('Lista:', '').strip()
        
        # 2. Manejar separadores: eliminar comas utilizadas como separadores de miles
        precio_limpio = precio_limpio.replace(',', '') 
        
        # 3. Forzar formato de un solo punto decimal, eliminando puntos adicionales
        if precio_limpio.count('.') > 1:
            partes = precio_limpio.rsplit('.', 1) 
            entero = partes[0].replace('.', '') 
            decimal = partes[1] if len(partes) > 1 else '00'
            precio_limpio = f"{entero}.{decimal}"
        
        # 4. Asegurar que no exista doble punto residual
        precio_limpio = precio_limpio.replace('..', '.')
        
        try:
            # 5. Conversión final a tipo float
            return float(precio_limpio.strip())
        except ValueError:
            return None
    return None

### FUNCIÓN DE SOLICITUD HTTP Y PARSEO DEL CONTENIDO HTML
Esta función centraliza el proceso de conexión a la página web, realizando la solicitud HTTP y transformando la respuesta en un objeto BeautifulSoup. De esta manera, se desacopla la lógica de descarga del contenido de la lógica de extracción de datos, facilitando la reutilización del código y el mantenimiento del sistema.

In [ ]:
# --- 2. FUNCIÓN DE SOLICITUD HTTP Y PARSEO ---
def obtener_contenido_pagina(url):
    """
    Realiza una solicitud HTTP GET a la URL indicada y devuelve el
    contenido HTML parseado como un objeto BeautifulSoup.

    Esta función incorpora manejo de errores y validación del código
    de estado HTTP, asegurando que solo se procese contenido válido
    durante la etapa de scraping.

    Parámetros:
    url (str): Dirección web de la página a consultar.

    Retorna:
    BeautifulSoup | None: Objeto BeautifulSoup con el HTML parseado
    o None si ocurre un error durante la solicitud.
    """
    try:
        response = requests.get(url, headers=HEADERS, timeout=15)
        
        # Mostrar el código de estado HTTP para fines de monitoreo
        print(f"    - Código de estado HTTP recibido: {response.status_code}") 
        
        # Lanza una excepción si la respuesta HTTP indica error
        response.raise_for_status() 

        # Parseo del contenido HTML con BeautifulSoup
        return BeautifulSoup(response.content, "html.parser")

    except requests.exceptions.RequestException as e:
        # Manejo de errores de conexión, timeout o respuesta inválida
        print(f"❌ Error al realizar la solicitud a {url}: {e}")
        return None

### FUNCIÓN DE EXTRACCIÓN, FILTRADO Y PROCESAMIENTO DE OFERTAS
Esta función se encarga de recorrer el contenido HTML previamente parseado de Amazon y extraer información relevante únicamente de productos que presentan una oferta real (precio anterior y precio actual). Se aplica un filtrado estricto para garantizar que los datos obtenidos sean consistentes y útiles para el análisis comparativo de precios.

In [ ]:
# --- 3. FUNCIÓN DE EXTRACCIÓN Y FILTRADO ESTRICTO ---
def extraer_datos_amazon_ofertas(soup):
    """
    Extrae información de productos en oferta desde el HTML de Amazon,
    aplicando un filtrado estricto para conservar únicamente aquellos
    productos que cuentan con nombre, precio anterior y precio actual.

    La función también calcula el porcentaje de descuento utilizando
    valores numéricos normalizados, garantizando consistencia para
    análisis posteriores.

    Parámetros:
    soup (BeautifulSoup): Objeto BeautifulSoup con el HTML parseado
    de la página de resultados de Amazon.

    Retorna:
    list: Lista de diccionarios con la información estructurada de
    cada producto válido encontrado.
    """
    datos_productos = []

    # Selector principal de contenedores de productos
    contenedores_productos = soup.find_all('div', {'data-component-type': 's-search-result'})
    
    # Mensaje informativo sobre la cantidad de productos encontrados
    print(f"    -> Productos encontrados para extraer: {len(contenedores_productos)}")

    for contenedor in contenedores_productos:
        
        # Inicialización de variables con valores por defecto
        nombre = 'N/A'
        precio_antes = 'N/A'
        precio_despues = 'N/A'
        url_image = 'N/A'
        
        # --- Extracción del nombre del producto ---
        titulo_h2 = contenedor.find('h2')
        if titulo_h2:
            span_titulo = titulo_h2.find('span')
            nombre = span_titulo.text.strip() if span_titulo else titulo_h2.text.strip()
        
        # --- Extracción del precio actual (precio_despues) ---
        precio_span = contenedor.find('span', class_='a-price')
        if precio_span:
            p_entero = precio_span.find('span', class_='a-price-whole')
            p_fraccion = precio_span.find('span', class_='a-price-fraction')
            moneda = precio_span.find('span', class_='a-price-symbol')
            
            p_entero_str = p_entero.text.strip() if p_entero else ''
            p_fraccion_str = p_fraccion.text.strip() if p_fraccion else ''
            moneda_str = moneda.text.strip() if moneda else 'USD'
            
            if p_entero_str or p_fraccion_str:
                # Construcción del precio en formato texto limpio
                precio_despues = f"{moneda_str} {p_entero_str}.{p_fraccion_str}".replace('\xa0', ' ').replace('..', '.') 

        # --- Extracción del precio anterior (precio_antes) ---
        precio_antes_tag = contenedor.find('span', class_='a-price', attrs={'data-a-strike': 'true'})
        
        if precio_antes_tag:
            offscreen_price = precio_antes_tag.find('span', class_='a-offscreen')
            if offscreen_price:
                precio_antes = offscreen_price.text.strip().replace('\u00a0', ' ')
            else:
                precio_antes = precio_antes_tag.text.strip().replace('\u00a0', ' ')
            
            # Limpieza adicional del texto del precio anterior
            precio_antes = precio_antes.replace('PVPR:', '').replace('Lista:', '').strip().replace('..', '.')
        
        # --- Extracción de la URL de la imagen del producto ---
        imagen_tag = contenedor.find('img', class_='s-image')
        url_image = imagen_tag.get('src') if imagen_tag and imagen_tag.get('src') else 'N/A'
        
        # --- Cálculo del descuento ---
        descuento = '0%'
        
        # Conversión de precios a formato numérico para el cálculo
        num_despues = limpiar_precio(precio_despues)
        num_antes = limpiar_precio(precio_antes) 
        
        if num_antes and num_despues and num_antes > num_despues:
            calc_descuento = ((num_antes - num_despues) / num_antes) * 100
            descuento = f"{calc_descuento:.0f}%" 
        
        # --- FILTRO ESTRICTO ---
        # Solo se agregan productos con nombre, precio actual y precio anterior válidos
        if nombre != 'N/A' and precio_despues != 'N/A' and precio_antes != 'N/A':
            datos_productos.append({
                "nombre": nombre.replace('\u00a0', ' '),
                "precio_antes": precio_antes.replace('\u00a0', ' '),
                "precio_despues": precio_despues.replace('\u00a0', ' '), 
                "descuento": descuento,
                "url_image": url_image
            })
        
    return datos_productos

### FUNCIÓN PRINCIPAL DE ORQUESTACIÓN Y PAGINACIÓN DINÁMICA
Esta función actúa como el controlador principal del scraper, coordinando la descarga de múltiples páginas de resultados, la extracción de datos de cada página y la navegación dinámica entre páginas mediante enlaces "Siguiente". Además, incorpora validaciones y mensajes de control para detectar errores críticos durante el proceso de scraping.

In [ ]:
# --- 4. ORQUESTACIÓN Y PAGINACIÓN DINÁMICA ---
def ejecutar_scraper_amazon_ofertas(url_inicial, num_paginas):
    """
    Ejecuta el proceso completo de scraping de ofertas desde Amazon,
    gestionando la paginación dinámica y centralizando la lógica de
    descarga, extracción y acumulación de los datos recolectados.

    Parámetros:
    url_inicial (str): URL inicial de búsqueda en Amazon.
    num_paginas (int): Número máximo de páginas a recorrer.

    Retorna:
    list: Lista consolidada de diccionarios con los datos de todas
    las ofertas válidas encontradas durante la ejecución.
    """
    total_datos = []
    url_actual = url_inicial
    
    # Bucle principal de paginación
    for pagina_actual in range(1, num_paginas + 1):
        if url_actual is None:
            print("⚠️ No se encontró el enlace a la página siguiente. Finalizando.")
            break
            
        # Mensaje informativo de progreso
        print(f"\n📢 Procesando página {pagina_actual}/{num_paginas}. URL actual: {url_actual}")
        
        # Obtención y parseo del contenido HTML
        soup = obtener_contenido_pagina(url_actual)
        
        if soup is None:
            print("🛑 Error al obtener la página. Deteniendo el scraper.")
            break
            
        # Extracción de datos de productos en oferta
        nuevos_datos = extraer_datos_amazon_ofertas(soup) 
        
        # Validación crítica en la primera página
        if not nuevos_datos and pagina_actual == 1:
            print("⚠️ ¡FALLO CRÍTICO! No se encontraron productos con oferta en la página 1.")
            break
        
        # Acumulación de los datos extraídos
        total_datos.extend(nuevos_datos)
        
        # Búsqueda del enlace a la siguiente página
        enlace_siguiente = soup.find('a', class_='s-pagination-next')

        if enlace_siguiente:
            url_actual = "https://www.amazon.com" + enlace_siguiente.get('href')
        else:
            url_actual = None 

        # Delay para respetar los servidores y evitar bloqueos
        time.sleep(3) 

    return total_datos

### EJECUCIÓN FINAL DEL SCRAPER Y PRESENTACIÓN DE RESULTADOS
Esta sección ejecuta el scraper completo cuando el archivo se ejecuta como programa principal. Además, se encarga de:
- Mostrar mensajes de estado del proceso
- Imprimir una muestra de los datos obtenidos
- Convertir los resultados a un DataFrame
- Guardar la información final en un archivo CSV para análisis posterior o trabajo colaborativo

In [ ]:
# --- 5. EJECUCIÓN E IMPRESIÓN COMO LISTA DE DICCIONARIOS ---
if __name__ == "__main__":
    
    # Ejecutar el proceso de extracción
    resultados_finales_diccionario = ejecutar_scraper_amazon_ofertas(URL_INICIAL, NUMERO_DE_PAGINAS)
    
    # ... (código de impresión y muestra) ...
    
    # Verificar si se obtuvieron resultados válidos
    if resultados_finales_diccionario:
        print(f"\n✅ Extracción de Amazon completada. Total de productos filtrados: {len(resultados_finales_diccionario)}.")
        
        # Imprimir la salida final en formato de lista de diccionarios
        print("\n--- SALIDA FINAL: LISTA DE DICCIONARIOS (Muestra de Ofertas Reales) ---")
        
        if len(resultados_finales_diccionario) > 0:
            # Imprimir solo una muestra para evitar saturar la consola
            print(json.dumps(resultados_finales_diccionario[:5], indent=4))
            print(f"\n... Se omiten {len(resultados_finales_diccionario) - 5} productos más para la vista previa. Total: {len(resultados_finales_diccionario)}.")
        
        # Conversión de los resultados a DataFrame para análisis
        df = pd.DataFrame(resultados_finales_diccionario)
        
        # Definición del nombre del archivo CSV de salida
        NOMBRE_ARCHIVO_CSV = 'amazon_ofertas_filtradas.csv'
        
        # Guardar los datos en formato CSV
        df.to_csv(NOMBRE_ARCHIVO_CSV, index=False, encoding='utf-8')
        print(f"\n💾 Datos guardados en CSV: {NOMBRE_ARCHIVO_CSV}")
        
    else:
        # Mensaje informativo cuando no se obtienen resultados
        print("\n⚠️ La extracción de Amazon no produjo resultados con los filtros aplicados.")